<center>به نام خدا</center>
<br>
<center> درس بازیابی پیشرفته اطلاعات - بهار ۱۴۰۰ - ۱۴۰۱ </center>
<br>
<center> تمرین پنجم - تحلیل لینک</center>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

address = "MIRNews1401"
import sys
sys.path.append('/content/drive/My Drive/{}'.format(address))
%cd /content/drive/My\ Drive/$address

Mounted at /content/drive
/content/drive/My Drive/MIRNews1401


In [ ]:
# !pip install hazm

In [ ]:
import json
import math
from tqdm import tqdm
# from hazm import *
import numpy as np

# 1. calculate similarities and adjancy graph

In [ ]:
news_details = np.load("./data/news_details.npy", allow_pickle=True)
transformer_doc_embeddings = np.load("./data/transformer_doc_embeddings.npy")

In [ ]:
new_doc_embeddings = np.array([i/np.linalg.norm(i) for i in transformer_doc_embeddings])

In [ ]:
similarity_matrix = np.dot(new_doc_embeddings, new_doc_embeddings.T) - np.identity(len(new_doc_embeddings))

In [ ]:
link_threshold = 0.8
similarity_np = similarity_matrix
similarity_np[similarity_np < link_threshold] = 0

In [ ]:
adj_matrix = np.ones((news_num, news_num))
adj_matrix[similarity_matrix < link_threshold] = 0

# 2. run algorithm

In [ ]:
from sklearn.preprocessing import normalize

<div dir=rtl>
ابتدا شباهت‌ها را به شکل نرم-1 در می‌آوریم تا مجموع احتمالات هر سطر برابر یک شود.
</div>

In [ ]:
similarity_normalized = normalize(similarity_np, norm='l1', axis=1)

In [ ]:
import networkx as nx

<div dir=rtl>
ماتریس نرمال شده را به یک گراف وزن‌دار تبدیل می‌کنیم و الگوریتم pagerank
را بر روی آن اجرا می‌کنیم. خروجی این الگوریتم یک دیکشنری است که به هر راس احتمال رسیدن به آن را مشخص می‌کند.
</div>

In [ ]:
G = nx.from_numpy_matrix(similarity_normalized)
pr = nx.pagerank(G, alpha=0.9, max_iter=100)
pr

{0: 0.0002292607768742439,
 1: 0.00025814564420570624,
 2: 0.0001644996239898756,
 3: 0.00019115085091873582,
 4: 9.99889491491866e-05,
 5: 0.00016598239135603852,
 6: 0.0001829710097881213,
 7: 0.00017018155805205947,
 8: 0.00017020940074327046,
 9: 0.00013516796687336653,
 10: 0.0002283065915540981,
 11: 0.00016652521911482768,
 12: 0.00020775185537592344,
 13: 0.00017880513239367491,
 14: 0.000188810037250611,
 15: 0.00031157844273366805,
 16: 0.0001854915856136823,
 17: 0.0001799800959447378,
 18: 0.00026304726705006563,
 19: 0.0001666266616741852,
 20: 0.00015459878793382782,
 21: 0.00018425219384350796,
 22: 0.0002801617406422149,
 23: 0.0001656066193343194,
 24: 0.00021371125536152395,
 25: 0.0001966873052523065,
 26: 0.00014631013822616437,
 27: 0.0001586339121510273,
 28: 0.00018534242981762327,
 29: 0.0001541955448741361,
 30: 0.000214988064636423,
 31: 0.0001786016860316067,
 32: 0.0002044348457579262,
 33: 0.0001711632389348076,
 34: 4.558636003883523e-05,
 35: 0.0003124429

In [ ]:
with open("./data/page_rank_result", 'w') as f:
        f.write(json.dumps(pr))

In [ ]:
with open("./data/page_rank_result") as f:
        new_pr = json.loads(f.read())

In [ ]:
number_of_largest = 10

In [ ]:
largest_pr_indices = sorted(pr, key=pr.get, reverse=True)[:number_of_largest]
for i in largest_pr_indices:
    print("probability:", format(pr[i], '.5f'), ", link:", news_details[i]['url'])

probability: 0.00050 , link: https://www.yjc.news/fa/news/7879378/
probability: 0.00048 , link: https://www.yjc.news/fa/news/7929153/
probability: 0.00048 , link: https://www.yjc.news/fa/news/7679483/
probability: 0.00046 , link: https://www.yjc.news/fa/news/7904138/
probability: 0.00045 , link: http://www.yjc.news/fa/news/7945139/گرانی-یقه-بازار-پوشاک-را-ول-نمی-کندگیاهی-از-سرزمین‌های-دوروعده-بی-سرانجام-واریزی
probability: 0.00045 , link: http://www.yjc.news/fa/news/7946559/گفتگوی-بی‌واسطه-رئیس‌جمهور-با-زلزله‌زدگانماجرای-توزیع-آردهای-حشره-زده-در-خوزستان-چه-بودراز-قتل-جسد‌های-سوخته-در-گرگان-کشف-شد
probability: 0.00045 , link: https://www.yjc.news/fa/news/7613734/
probability: 0.00044 , link: https://www.yjc.news/fa/news/7663093/
probability: 0.00044 , link: https://www.yjc.news/fa/news/7974077/قهر-آسمان-خشکی-زمین-راهکار‌های-رئیس-جمهور-برای-حل-مشکلات-آب-اصفهان-چیست-انتظار-طولانی-کرمانی‌ها-در-برزخِ-بهشت‌کریمان
probability: 0.00044 , link: https://www.yjc.news/fa/news/7770475/


<div dir=rtl>
در ادامه بر روی ماتریس مجاورت الگوریتم HITS
را اجرا می‌کنیم. خروجی آن دو دیکشنری برای hubs
و authorities است که احتمال هر راس در این مجموعه‌ها را نشان می‌دهد.
</div>

In [ ]:
G2 = nx.from_numpy_matrix(adj_matrix)
hubs, authorities = nx.hits(G2, max_iter = 100, normalized = True)

In [ ]:
with open("./data/hits_hubs_result", 'w') as f:
        f.write(json.dumps(hubs))
with open("./data/hits_auth_result", 'w') as f:
        f.write(json.dumps(authorities))

<div dir=rtl>
در دو قسمت زیر، 10 تا از فایل‌ها با بیشترین مقدار hub،
و 10 تا از فایل‌ها با بیشترین مقدار authority مشخص شده‌اند.
</div>

In [ ]:
largest_hubs_indices = sorted(hubs, key=hubs.get, reverse=True)[:number_of_largest]
for i in largest_hubs_indices:
    print("hub score:", format(hubs[i], '.5f'), ", link:", news_details[i]['url'])

hub score: 0.00080 , link: https://www.yjc.news/fa/news/7679483/
hub score: 0.00079 , link: https://www.yjc.news/fa/news/8127073/درخواست-متقاضیان-برای-تعویق-آزمون-ارشد-و-گلایه-از-بسته-شدن-بورس-در-بانک-دی
hub score: 0.00079 , link: https://www.yjc.news/fa/news/7879378/
hub score: 0.00078 , link: https://www.yjc.news/fa/news/8135091
hub score: 0.00078 , link: https://www.yjc.news/fa/news/7929153/
hub score: 0.00078 , link: https://www.yjc.news/fa/news/8120985/بلاتکلیفی-متقاضیان-کنکور-۱۴۰۲-و-تاخیر-در-پلاک-خودروی-رانا-پلاس
hub score: 0.00078 , link: https://www.yjc.news/fa/news/8181830/گلایه-از-پلاک-نشدن-تاکسی-سمند-و-خشکسالی-درختان-بلوط-در-شهرستان-رستم
hub score: 0.00078 , link: https://www.yjc.news/fa/news/7929915/
hub score: 0.00077 , link: http://www.yjc.news/fa/news/7947531/امید‌ها-برای-نجات-یوز-ایرانی-زنده-شدارادت-به-متوفی-به-سنگ-قبر-گران-استحقوق-کارگران-در-جیب-کدام-مسئولان-هلهله-می‌کنند
hub score: 0.00077 , link: http://www.yjc.news/fa/news/7946559/گفتگوی-بی‌واسطه-رئیس‌جمهور-با-زلزله

In [ ]:
largest_authorities_indices = sorted(hubs, key=authorities.get, reverse=True)[:number_of_largest]
for i in largest_authorities_indices:
    print("authority score:", format(authorities[i], '.5f'), ", link:", news_details[i]['url'])

authority score: 0.00080 , link: https://www.yjc.news/fa/news/7679483/
authority score: 0.00079 , link: https://www.yjc.news/fa/news/8127073/درخواست-متقاضیان-برای-تعویق-آزمون-ارشد-و-گلایه-از-بسته-شدن-بورس-در-بانک-دی
authority score: 0.00079 , link: https://www.yjc.news/fa/news/7879378/
authority score: 0.00078 , link: https://www.yjc.news/fa/news/8135091
authority score: 0.00078 , link: https://www.yjc.news/fa/news/7929153/
authority score: 0.00078 , link: https://www.yjc.news/fa/news/8120985/بلاتکلیفی-متقاضیان-کنکور-۱۴۰۲-و-تاخیر-در-پلاک-خودروی-رانا-پلاس
authority score: 0.00078 , link: https://www.yjc.news/fa/news/8181830/گلایه-از-پلاک-نشدن-تاکسی-سمند-و-خشکسالی-درختان-بلوط-در-شهرستان-رستم
authority score: 0.00078 , link: https://www.yjc.news/fa/news/7929915/
authority score: 0.00077 , link: http://www.yjc.news/fa/news/7947531/امید‌ها-برای-نجات-یوز-ایرانی-زنده-شدارادت-به-متوفی-به-سنگ-قبر-گران-استحقوق-کارگران-در-جیب-کدام-مسئولان-هلهله-می‌کنند
authority score: 0.00077 , link: http://www.